# Generate isochrones by state
To determine the driving distance from each dealer, we used the open route service's isochrone feature. It is documented here. We did each state individually to speed up processing time. 

In [1]:
import pandas as pd
import requests
import json

In [2]:
# docker compose up
# https://download.geofabrik.de/north-america/us.html

In [4]:
ffls = pd.read_csv("../../data/processed/atf-ffl-list/ffls_2023.csv")

In [5]:
# ffls.head()

In [330]:
state_ffls = ffls[ffls["state_name"] == "New Mexico"]

In [331]:
len(state_ffls)

750

In [332]:
# state_ffls.iloc[0:10]

In [333]:
target = ['NM']
abbr = 'nm'
state = 'new-mexico'
distance = 600 # 10 minutes (600), 20 minutes (1200), 30 minutes (1800)
mvmt_type = 'driving'

In [334]:
# Create empty feature collection
collection = {'type': 'FeatureCollection', 'features': []}
no_iso = pd.DataFrame()
i = 0

for row in state_ffls.itertuples(index=False):
    print(i, row.original_address)
    try:
        # Set variables
        request_locations = [[row.longitude,row.latitude]]
        request_range = [distance]
        body = {
            "locations": request_locations,
            "range": request_range,
            "location_type": "start",
            "range_type": "time",
        }
        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Content-Type': 'application/json; charset=utf-8'
        }
        # Call the request
        # call = requests.post('http://localhost:8080/ors/v2/isochrones/foot-walking', json=body, headers=headers)
        call = requests.post('http://localhost:8080/ors/v2/isochrones/driving-car', json=body, headers=headers)
        # Pull out the feature
        data = json.loads(call.text)
        if "features" in data:
            feature = data["features"][0]
            # Add the FFL id
            feature["properties"]["original_address"] = row.original_address
            # Append to the feature collection
            collection["features"].append(feature)
        else:
            no_iso = pd.concat([no_iso, state_ffls.iloc[[i]]])
    except:
        no_iso = pd.concat([no_iso, state_ffls.iloc[[i]]])
    
    i = i + 1

0 617 N 4TH ST, BELEN, NM, 87002
1 1 ESTANCIA ROAD, BELEN, NM, 87002
2 1276 HWY 116, BELEN, NM, 87002
3 19723 HIGHWAY 314, BELEN, NM, 87002
4 400 RIO COMMUNITIES BLVD, BELEN, NM, 87002
5 1621 MESITA LN, BELEN, NM, 87002
6 1602 TOMMY ARMOUR CT, BELEN, NM, 87002
7 1602 TOMMY ARMOUR CT, BELEN, NM, 87002
8 80 DON LUIS TRUJILLO BLVD  OFFICE 24, BELEN, NM, 87002
9 64 FAWN CIRCLE, BELEN, NM, 87002
10 1307 WARRIOR WAY, SANTA ANA PUEBLO, NM, 87004
11 1083 CAMINO DEL BOSQUE, BERNALILLO, NM, 87004
12 348 CAMINO DEL PUEBLO, BERNALILLO, NM, 87004
13 62 CEMETERY RD, BLUEWATER, NM, 87005
14 62 CEMETERY RD, BLUEWATER, NM, 87005
15 62 CEMETERY RD, BLUEWATER, NM, 87005
16 12 SPRING CT, EDGEWOOD, NM, 87015
17 5 OLIVER CT, EDGEWOOD, NM, 87015
18 72 SANDOVAL ROAD, EDGEWOOD, NM, 87015
19 44 KENNEDY RD, EDGEWOOD, NM, 87015
20 1912 OLD HWY 66, EDGEWOOD, NM, 87015
21 37 MADOLE RD, EDGEWOOD, NM, 87015
22 96 SANTA MARIA DR, EDGEWOOD, NM, 87015
23 4 PINTO CT, EDGEWOOD, NM, 87015
24 1890 OLD  HWY 66, EDGEWOOD, NM,

In [335]:
len(collection.get('features'))

750

In [336]:
with open('../../data/processed/isochrones/'+abbr+'_'+str(distance)+'_'+mvmt_type+'.geojson', 'w') as f:
    json.dump(collection, f)